# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## GPU Check

First, you'll need to enable GPUs for the notebook:

- Navigate to `Edit`→`Notebook Settings`
- Select T4 GPU from the Hardware Accelerator section
- Click `Save` and accept.

Next, we'll confirm that we can connect to the GPU:

In [2]:
import torch

if not torch.cuda.is_available():
    raise RuntimeError("GPU not available")
else:
    print("GPU is available!")

GPU is available!


## Installing dependencies

In [3]:
%pip install ragatouille PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.9/490.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 40.1 MB/s eta 0:00:00
  

In [4]:
%pip install git+https://github.com/mozilla-ai/structured-qa.git@5-add-benchmark

  Cloning https://github.com/mozilla-ai/structured-qa.git (to revision 5-add-benchmark) to /tmp/pip-req-build-49ruike5
  Running command git clone --filter=blob:none --quiet https://github.com/mozilla-ai/structured-qa.git /tmp/pip-req-build-49ruike5
  Running command git checkout -b 5-add-benchmark --track origin/5-add-benchmark
  Switched to a new branch '5-add-benchmark'
  Branch '5-add-benchmark' set up to track remote branch '5-add-benchmark' from 'origin'.
  Resolved https://github.com/mozilla-ai/structured-qa.git to commit 17942ca192e0493c7c061e6f908cc2b945122ef6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 MB 13.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done

In [5]:
!wget https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv

--2025-01-24 10:32:07--  https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14711 (14K) [text/plain]
Saving to: ‘structured_qa.csv’

structured_qa.csv   100%[===================>]  14.37K  --.-KB/s    in 0s      

2025-01-24 10:32:07 (73.1 MB/s) - ‘structured_qa.csv’ saved [14711/14711]



# Setup

In [6]:
import os
import google.generativeai as genai
from google.colab.userdata import get, SecretNotFoundError

try:
    genai.configure(api_key=get("GOOGLE_API_KEY"))
except SecretNotFoundError as e:
    raise RuntimeError("Please set the GOOGLE_API_KEY secret to your API key") from e
os.environ["LOGURU_LEVEL"] = "INFO"

In [7]:
from loguru import logger

In [10]:
import PyPDF2


def load_pdf(pdf_file: str) -> str | None:
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        return "\n".join(page.extract_text() for page in pdf_reader.pages)
    except Exception as e:
        logger.exception(e)
        return None

## Function to Process all questions for a single Document

In [47]:
import json
import time

from ragatouille import RAGPretrainedModel
from ragatouille.data import CorpusProcessor


def process_document(
    document_file,
    document_data,
    model,
):
    logger.info("Setting up RAG")
    RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
    corpus_processor = CorpusProcessor()
    documents = corpus_processor.process_corpus([load_pdf(document_file)])
    RAG.encode([x["content"] for x in documents])

    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in document_data.iterrows():
        if model.n > 0 and model.n % 9 == 0:
            logger.info("Waiting for 60 seconds")
            time.sleep(60)
        question = row["question"]

        logger.info(f"Question: {question}")
        results = RAG.search_encoded_docs(query=question, k=3)
        current_info = "\n".join(result["content"] for result in results)
        logger.info(current_info[:100])

        answer = model.model.generate_content(
            [f"This is the document: {current_info}", question]
        )
        logger.info(answer.text)
        answers[index] = json.loads(answer.text)["answer"]
        sections[index] = None
        model.n += 1

    return answers, sections

## Load Model

In [48]:
from structured_qa.model_loaders import load_gemini_model

In [49]:
SYSTEM_PROMPT = """
You are given an input document and a question.
You can only answer the question based on the information in the document.
You will return a JSON name with a single key: "answer".
In `"answer"`, you will return the answer using one of the following JSON types:
- Yes/No (for boolean questions)
Is the model an LLM?
{
  "answer": "No"
}
- Single number (for numeric questions)
How many layers does the model have?
{
  "answer": 12
}
- Single letter (for multiple-choice questions)
What is the activation function used in the model?
-A: ReLU
-B: Sigmoid
-C: Tanh
{
  "answer": "C"
}
"""

In [50]:
model = load_gemini_model(
    "gemini-2.0-flash-exp",
    system_prompt=SYSTEM_PROMPT,
    generation_config={
        "response_mime_type": "application/json",
    },
)
model.n = 0

# Run Benchmark

In [51]:
from pathlib import Path
from urllib.request import urlretrieve

import pandas as pd

logger.info("Loading input data")
data = pd.read_csv("structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)
for document_link, document_data in data.groupby("document"):
    logger.info(f"Downloading document {document_link}")
    downloaded_document = Path(f"{Path(document_link).name}.pdf")
    if not Path(downloaded_document).exists():
        urlretrieve(document_link, downloaded_document)
        logger.info(f"Downloaded {document_link} to {downloaded_document}")
    else:
        logger.info(f"File {downloaded_document} already exists")

    answers, sections = process_document(downloaded_document, document_data, model)

    for index in document_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-01-24 11:10:27.632 | INFO     | __main__:<cell line: 0>:8 - Loading input data
2025-01-24 11:10:27.640 | INFO     | __main__:<cell line: 0>:13 - Downloading document https://arxiv.org/pdf/1706.03762
2025-01-24 11:10:27.641 | INFO     | __main__:<cell line: 0>:19 - File 1706.03762.pdf already exists
2025-01-24 11:10:27.643 | INFO     | __main__:process_document:13 - Setting up RAG
/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


Encoding 56 documents...


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
100%|██████████| 2/2 [00:00<00:00,  7.78it/s]
2025-01-24 11:10:30.151 | INFO     | __main__:process_document:19 - Predicting
2025-01-24 11:10:30.157 | INFO     | __main__:process_document:28 - Question: What type of architecture does the model use? -A: decoder only -B: encoder only -C: encoder-decoder
2025-01-24 11:10:30.174 | INFO     | __main__:process_document:31 - (2015) [23] multi-task 93.0
Dyer et al. (2016) [8] generative 93.3
increased the maximum output leng


Shapes:
encodings: torch.Size([56, 508, 128])
doc_masks: torch.Size([56, 508])
Documents encoded!


2025-01-24 11:10:31.916 | INFO     | __main__:process_document:34 - {
"answer": "C"
}
2025-01-24 11:10:31.918 | INFO     | __main__:process_document:28 - Question: How many layers compose the encoder?
2025-01-24 11:10:31.941 | INFO     | __main__:process_document:31 - At each step the model is auto-regressive
[10], consuming the previously generated symbols as additi
2025-01-24 11:10:33.208 | INFO     | __main__:process_document:34 - {
  "answer": 6
}
2025-01-24 11:10:33.209 | INFO     | __main__:process_document:28 - Question: How many layers compose the decoder?
2025-01-24 11:10:33.225 | INFO     | __main__:process_document:31 - To facilitate these residual connections, all sub-layers in the model, as well as the embedding
laye
2025-01-24 11:10:34.413 | INFO     | __main__:process_document:34 - {
  "answer": 6
}
2025-01-24 11:10:34.415 | INFO     | __main__:process_document:28 - Question: How many parallel attention heads are used?
2025-01-24 11:10:34.430 | INFO     | __main__:proces

...

2025-01-24 11:17:25.988 | INFO     | __main__:process_document:34 - {
"answer": 3
}
2025-01-24 11:17:25.990 | INFO     | __main__:process_document:28 - Question: How many victory conditions are there?
2025-01-24 11:17:26.006 | INFO     | __main__:process_document:31 - Conquering Middle-earth
If you are present in all 7 regions (with a Fortress and/or at least 1 Unit)
2025-01-24 11:17:27.345 | INFO     | __main__:process_document:34 - {
  "answer": 3
}
2025-01-24 11:17:27.346 | INFO     | __main__:process_document:28 - Question: How many different races are there?
2025-01-24 11:17:27.362 | INFO     | __main__:process_document:31 - • 
 Re
veal tokens to both players. There is no hidden information.Bonus spaces
2 matching Race symb
2025-01-24 11:17:28.651 | INFO     | __main__:process_document:34 - {
  "answer": 6
}
2025-01-24 11:17:28.652 | INFO     | __main__:process_document:24 - Waiting for 60 seconds
2025-01-24 11:18:28.654 | INFO     | __main__:process_document:28 - Question: Which 

Encoding 48 documents...


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
100%|██████████| 2/2 [00:00<00:00, 10.41it/s]
2025-01-24 11:19:49.710 | INFO     | __main__:process_document:19 - Predicting
2025-01-24 11:19:49.712 | INFO     | __main__:process_document:28 - Question: What is the maximum number of cards a player may acquire during the lookout phase?
2025-01-24 11:19:49.729 | INFO     | __main__:process_document:31 - 13. At the beginning of the game, each player draws 5 cards from their 
Clan deck, looks through the


Shapes:
encodings: torch.Size([48, 508, 128])
doc_masks: torch.Size([48, 508])
Documents encoded!


2025-01-24 11:19:51.094 | INFO     | __main__:process_document:34 - {
  "answer": 4
}
2025-01-24 11:19:51.096 | INFO     | __main__:process_document:28 - Question: Is there a limit to the number of cards a player may have in their hand?
2025-01-24 11:19:51.112 | INFO     | __main__:process_document:31 - NOTE 1:  There’s no limit to the number of cards a player may have 
in their hand.  
NOTE 2:  If the
2025-01-24 11:19:52.626 | INFO     | __main__:process_document:34 - {
  "answer": "No"
}
2025-01-24 11:19:52.628 | INFO     | __main__:process_document:28 - Question: Can you raid the locations of a player that has passed during the action phase?
2025-01-24 11:19:52.645 | INFO     | __main__:process_document:31 - Starting with the First player and continuing clockwise, 
each player performs one action at a time.
2025-01-24 11:19:53.958 | INFO     | __main__:process_document:34 - {
  "answer": "No"
}
2025-01-24 11:19:53.959 | INFO     | __main__:process_document:28 - Question: Can players

In [53]:
results = pd.read_csv("results.csv")
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,section,question,answer,pred_answer,pred_section
12,12,https://arxiv.org/pdf/2210.05189,2.1 Fully Connected Networks,Does the model use Sigmoid activation function?,NO,YES,NaN
22,22,https://eur-lex.europa.eu/legal-content/EN/TXT...,Prohibited AI Practices (Article 5),Which type of AI systems are banned by the AI ...,C,B,NaN
24,24,https://eur-lex.europa.eu/legal-content/EN/TXT...,Classification rules (article 51),"What is the threshold, measured in floating po...",C,NOT APPLICABLE,NaN
50,50,https://github.com/mozilla-ai/structured-qa/re...,CHAPTER OVERVIEW,Which player begins the game? -A: Sauron -B: T...,A,C,NaN


In [54]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.9473684210526315